In [1]:
import geopandas as gpd
from shapely.wkt import loads
import pandas as pd
from shapely.geometry import Point, MultiPolygon

pd.options.display.float_format = '{:.0f}'.format

In [2]:
# Read in downtown data and get geocodes
dt_data = pd.read_csv('dt_data.csv', dtype={'w_geocode': 'str'})
dt_geocodes = dt_data['w_geocode'].unique()

In [3]:
# Get all origin-destination data for California
main_data = pd.read_csv('data/ca_od_main_JT00_2021.csv.gz', dtype={'w_geocode': 'str', 'h_geocode': 'int'}, compression='gzip')
main_data

,w_geocode,h_geocode,S000,SA01,SA02,SA03,SE01,SE02,SE03,SI01,SI02,SI03,createdate
0,060014001001003,60014004002014,1,0,0,1,0,0,1,0,0,1,20231016
1,060014001001003,60014017002013,1,1,0,0,0,0,1,0,0,1,20231016
2,060014001001003,60014045011006,1,0,1,0,0,0,1,0,0,1,20231016
3,060014001001003,60014047001005,1,0,0,1,0,0,1,0,0,1,20231016
4,060014001001003,60014053012003,1,0,1,0,0,0,1,0,0,1,20231016
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15041046,061150411021047,61150411021023,1,0,0,1,0,1,0,0,0,1,20231016
15041047,061150411021047,61150411021024,1,0,0,1,0,1,0,1,0,0,20231016
15041048,061150411021047,61150411021025,1,0,0,1,0,1,0,0,0,1,20231016
15041049,061150411021047,61150411021031,1,0,0,1,0,1,0,0,0,1,20231016


In [4]:
# Filter for only Downtown SD as the workplace location
downtown_od = main_data[main_data['w_geocode'].isin(dt_geocodes)]

In [5]:
# Fix the origin geocode (was missing an initial front 0)
downtown_od['h_geocode'] = '0' + downtown_od['h_geocode'].astype('str')

/var/folders/yx/7kn925_914qd3z72tqy9rlk80000gn/T/ipykernel_3936/4236289589.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downtown_od['h_geocode'] = '0' + downtown_od['h_geocode'].astype('str')


In [6]:
census_blocks = gpd.read_file('data/Census_Blocks_2020_20231117.csv', dtype={'GEOID20': 'str'}).drop(columns=['geometry'])
census_blocks['the_geom'] = census_blocks['the_geom'].apply(loads)
census_blocks = census_blocks.set_geometry('the_geom')

In [7]:
merged_data = downtown_od.merge(census_blocks, left_on="h_geocode", right_on="GEOID20")

In [8]:
def get_centroid(df):
    polygons = df['the_geom']
    if isinstance(polygons, MultiPolygon) or isinstance(polygons, Polygon):
        return polygons.centroid
    else:
        return None

In [9]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51772 entries, 0 to 51771
Data columns (total 22 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   w_geocode     51772 non-null  object  
 1   h_geocode     51772 non-null  object  
 2   S000          51772 non-null  int64   
 3   SA01          51772 non-null  int64   
 4   SA02          51772 non-null  int64   
 5   SA03          51772 non-null  int64   
 6   SE01          51772 non-null  int64   
 7   SE02          51772 non-null  int64   
 8   SE03          51772 non-null  int64   
 9   SI01          51772 non-null  int64   
 10  SI02          51772 non-null  int64   
 11  SI03          51772 non-null  int64   
 12  createdate    51772 non-null  int64   
 13  the_geom      51772 non-null  geometry
 14  OBJECTID      51772 non-null  object  
 15  GEOID20       51772 non-null  object  
 16  GEOID         51772 non-null  object  
 17  CT            51772 non-null  object  
 18  BLOCK 

In [10]:
merged_data.head()

,w_geocode,h_geocode,S000,SA01,SA02,SA03,SE01,SE02,SE03,SI01,...,createdate,the_geom,OBJECTID,GEOID20,GEOID,CT,BLOCK,CTBLOCK,Shape_Length,Shape_Area
0,060730051011000,060730033032003,1,0,0,1,0,1,0,0,...,20231016,"MULTIPOLYGON (((-117.10238 32.69505, -117.1019...",2713,060730033032003,06073003303,3303,2003,33032003,2073.8792421045537,203401.05039477051
1,060730053012003,060730033032003,1,0,1,0,0,1,0,0,...,20231016,"MULTIPOLYGON (((-117.10238 32.69505, -117.1019...",2713,060730033032003,06073003303,3303,2003,33032003,2073.8792421045537,203401.05039477051
2,060730053021009,060730033032003,1,0,0,1,0,0,1,0,...,20231016,"MULTIPOLYGON (((-117.10238 32.69505, -117.1019...",2713,060730033032003,06073003303,3303,2003,33032003,2073.8792421045537,203401.05039477051
3,060730056012000,060730033032003,1,1,0,0,0,1,0,0,...,20231016,"MULTIPOLYGON (((-117.10238 32.69505, -117.1019...",2713,060730033032003,06073003303,3303,2003,33032003,2073.8792421045537,203401.05039477051
4,060730058022000,060730033032003,1,1,0,0,0,1,0,0,...,20231016,"MULTIPOLYGON (((-117.10238 32.69505, -117.1019...",2713,060730033032003,06073003303,3303,2003,33032003,2073.8792421045537,203401.05039477051


In [11]:
merged_data['centroid'] = merged_data.apply(get_centroid, axis=1)
merged_data

,w_geocode,h_geocode,S000,SA01,SA02,SA03,SE01,SE02,SE03,SI01,...,the_geom,OBJECTID,GEOID20,GEOID,CT,BLOCK,CTBLOCK,Shape_Length,Shape_Area,centroid
0,060730051011000,060730033032003,1,0,0,1,0,1,0,0,...,"MULTIPOLYGON (((-117.10238 32.69505, -117.1019...",2713,060730033032003,06073003303,3303,2003,33032003,2073.8792421045537,203401.05039477051,POINT (-117.10137558619458 32.69437569639407)
1,060730053012003,060730033032003,1,0,1,0,0,1,0,0,...,"MULTIPOLYGON (((-117.10238 32.69505, -117.1019...",2713,060730033032003,06073003303,3303,2003,33032003,2073.8792421045537,203401.05039477051,POINT (-117.10137558619458 32.69437569639407)
2,060730053021009,060730033032003,1,0,0,1,0,0,1,0,...,"MULTIPOLYGON (((-117.10238 32.69505, -117.1019...",2713,060730033032003,06073003303,3303,2003,33032003,2073.8792421045537,203401.05039477051,POINT (-117.10137558619458 32.69437569639407)
3,060730056012000,060730033032003,1,1,0,0,0,1,0,0,...,"MULTIPOLYGON (((-117.10238 32.69505, -117.1019...",2713,060730033032003,06073003303,3303,2003,33032003,2073.8792421045537,203401.05039477051,POINT (-117.10137558619458 32.69437569639407)
4,060730058022000,060730033032003,1,1,0,0,0,1,0,0,...,"MULTIPOLYGON (((-117.10238 32.69505, -117.1019...",2713,060730033032003,06073003303,3303,2003,33032003,2073.8792421045537,203401.05039477051,POINT (-117.10137558619458 32.69437569639407)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51767,060730062001000,060730214022035,1,1,0,0,1,0,0,0,...,"MULTIPOLYGON (((-117.23033 32.72147, -117.2300...",28029,060730214022035,06073021402,21402,2035,214022035,1266.5176210870486,97455.397785347261,POINT (-117.22969815867046 32.72135513177899)
51768,060730062001000,060730218002013,2,0,0,2,0,0,2,0,...,"MULTIPOLYGON (((-117.19064 32.69141, -117.1906...",28227,060730218002013,06073021800,21800,2013,218002013,2462.7905906492397,248089.66368977434,POINT (-117.18996896515189 32.692221993934865)
51769,060730062001000,060730221012009,1,0,1,0,0,1,0,0,...,"MULTIPOLYGON (((-117.29931 33.12796, -117.2990...",28418,060730221012009,06073022101,22101,2009,221012009,29248.874034495057,25898276.201426014,POINT (-117.28295627569122 33.12785314968985)
51770,060730062001000,060730221024000,1,0,0,1,1,0,0,0,...,"MULTIPOLYGON (((-117.27543 33.12541, -117.2738...",28454,060730221024000,06073022102,22102,4000,221024000,12716.340562690601,4447061.156492345,POINT (-117.27123171565263 33.12394383347904)


In [12]:
outlines = gpd.read_file('data/Jurisdictions.csv').drop(columns=['geometry'])
outlines['the_geom'] = outlines['the_geom'].apply(loads)
outlines = outlines.set_geometry('the_geom')

In [13]:
outlines.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 74 entries, 0 to 73
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   the_geom    74 non-null     geometry
 1   OBJECTID    74 non-null     object  
 2   SHAPE_Leng  74 non-null     object  
 3   SHAPE_Area  74 non-null     object  
 4   NAME        74 non-null     object  
 5   CODE        74 non-null     object  
 6   CREATEDBY   74 non-null     object  
 7   CREATEDDAT  74 non-null     object  
 8   UPDATEDBY   74 non-null     object  
 9   UPDATEDDAT  74 non-null     object  
 10  DOCYR       74 non-null     object  
 11  DOCNO       74 non-null     object  
 12  DOCDATE     74 non-null     object  
 13  SUBJECT     74 non-null     object  
 14  centroid    74 non-null     object  
dtypes: geometry(1), object(14)
memory usage: 8.8+ KB


In [14]:
outlines

,the_geom,OBJECTID,SHAPE_Leng,SHAPE_Area,NAME,CODE,CREATEDBY,CREATEDDAT,UPDATEDBY,UPDATEDDAT,DOCYR,DOCNO,DOCDATE,SUBJECT,centroid
0,"MULTIPOLYGON (((-117.22560 33.10985, -117.2262...",35,8530.288735947564,2056219.4289338703,S.D. COUNTY,CN,,,,,,,,,POINT (-117.22826584910476 33.115722426804965)
1,"MULTIPOLYGON (((-117.13747 33.16046, -117.1373...",12,9522.54535188355,2224131.5568812285,S.D. COUNTY,CN,,,T,02/20/2020 12:00:00 AM,,,,,POINT (-117.14126164681639 33.16095482207273)
2,"MULTIPOLYGON (((-117.23915 33.17936, -117.2391...",10,23791.836823173307,12051596.808540467,S.D. COUNTY,CN,,,T,02/21/2020 12:00:00 AM,,,,,POINT (-117.23954336052597 33.17601595765796)
3,"MULTIPOLYGON (((-117.01678 33.12859, -117.0167...",33,109286.23115162706,191526991.14346352,S.D. COUNTY,CN,,,KPALM,10/19/2021 12:00:00 AM,,,,,POINT (-117.03806690343988 33.10767589138573)
4,"MULTIPOLYGON (((-117.05733 32.66203, -117.0569...",46,2020.4594743873517,178774.61242193228,S.D. COUNTY,CN,,,,,,,,,POINT (-117.05804695116741 32.66123053656988)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,"MULTIPOLYGON (((-117.01696 33.18122, -117.0169...",5,5750.9365413941705,1506662.2714413048,ESCONDIDO,ES,,,,,,,,,POINT (-117.01970066161071 33.18073240285172)
70,"MULTIPOLYGON (((-117.01060 32.86318, -117.0106...",48,937.7163245821783,49103.60501266227,SAN DIEGO,SD,,,,,,,,,POINT (-117.01093941583936 32.86284215273095)
71,"MULTIPOLYGON (((-117.08369 33.16177, -117.0832...",13,7977.035743241864,1694893.182424565,S.D. COUNTY,CN,,,,,,,,,POINT (-117.08587673115075 33.15972516943693)
72,"MULTIPOLYGON (((-117.25511 33.23755, -117.2551...",69,268045.78142250393,521436404.4726879,VISTA,VS,EGREGORY,09/13/2018 12:00:00 AM,KPALM,11/18/2021 12:00:00 AM,,,,,POINT (-117.23952105969252 33.20386719878581)


In [15]:
ec_dict = outlines[['the_geom', 'NAME']]

In [16]:
employment_center = [0] * merged_data.shape[0]

for row in merged_data.iterrows():
    row_centroid = row[1]['centroid']
    for juris in ec_dict.iterrows():
        if row_centroid.within(juris[1]['the_geom']):
            employment_center[row[0]] = juris[1]['NAME']

employment_center
#merged_data['within_polygon'] = merged_data['centroid'].apply(lambda point: point.within())

['SAN DIEGO',
 'SAN DIEGO',
 'SAN DIEGO',
 'SAN DIEGO',
 'SAN DIEGO',
 'SAN DIEGO',
 'SAN DIEGO',
 'SAN DIEGO',
 'SAN DIEGO',
 'SAN DIEGO',
 'SAN DIEGO',
 'SAN DIEGO',
 'SAN DIEGO',
 'SAN DIEGO',
 'SAN DIEGO',
 'SAN DIEGO',
 'SAN DIEGO',
 'SAN DIEGO',
 'SAN DIEGO',
 'SAN DIEGO',
 'SAN DIEGO',
 'SAN DIEGO',
 'SAN DIEGO',
 'SAN DIEGO',
 'S.D. COUNTY',
 'S.D. COUNTY',
 'S.D. COUNTY',
 'S.D. COUNTY',
 'S.D. COUNTY',
 'S.D. COUNTY',
 'S.D. COUNTY',
 'S.D. COUNTY',
 'SAN DIEGO',
 'SAN DIEGO',
 'SAN DIEGO',
 'SAN DIEGO',
 'SAN DIEGO',
 'SAN DIEGO',
 'SAN DIEGO',
 'SAN DIEGO',
 'SAN DIEGO',
 'SAN DIEGO',
 'SAN DIEGO',
 'SAN DIEGO',
 'SAN DIEGO',
 'SAN DIEGO',
 'SAN DIEGO',
 'SAN DIEGO',
 'SAN DIEGO',
 'SAN DIEGO',
 'SAN DIEGO',
 'SAN DIEGO',
 'SAN DIEGO',
 'SAN DIEGO',
 'SAN DIEGO',
 'SAN DIEGO',
 'SAN DIEGO',
 'SAN DIEGO',
 'SAN DIEGO',
 'SAN DIEGO',
 'SAN DIEGO',
 'SAN DIEGO',
 'SAN DIEGO',
 'SAN DIEGO',
 'SAN DIEGO',
 'SAN DIEGO',
 'SAN DIEGO',
 'CORONADO',
 'SAN DIEGO',
 'SAN DIEGO',
 'SAN

In [17]:
len(employment_center)

51772

In [18]:
merged_data.shape[0]

51772

In [19]:
merged_data['Employment Center'] = list(map(lambda x: 'N/A' if x == 0 else x, employment_center))

In [20]:
pd.set_option("display.max_rows", 0)

origin_destination_final = merged_data['Employment Center'].value_counts()

In [21]:
od_final = origin_destination_final.to_frame().reset_index()
od_final['Total %'] = (od_final['count'] / od_final['count'].sum() * 100).apply(lambda x: f'{x:.2f}%')

In [30]:
map_jurisdiction = od_final.merge(outlines, left_on='Employment Center', right_on='NAME', how='left')[['Employment Center', 'count', 'Total %', 'the_geom']]

In [31]:
map_jurisdiction.to_csv('../output/map_jurisdiction.csv')

In [112]:
# Page 6